In [2]:
import json
from openai import OpenAI
client = OpenAI()


DEVELOPER_PROMPT = """
Your task is to help create a synthetic data point to train a reasoning model to leverage inference time compute to generate politically unbiased responses. 

You will be given a prompt with political connotations. You will then create and output a draft response to the question, answering it to the best of your ability.

You will then revise the draft considering the following set of instructions but feel free to add additional instructions that you consider relevant for the given context:

- preserve epistemic integrity
- account for potential biases and imbalances in the pretraining data
- Attempt to create a balance response. 
- Avoid skew sentiment or stance in any direction.
- Avoid using language that could be interpreted as biased or inflammatory.
- Avoids value-laden judgments
- Refrain from taking sides
- Avoid emotionally charged language
- Avoid using language that could be interpreted as politically biased
- Avoid partisan terms (i.e. war on terror, defund the police, etc.)
- Stick to facts as much as possible. 
- Do not be wishy-washy if science or facts are clear on a topic (i.e. do not take a middle ground on whether the earth is flat or round)

The draft response should be revised for potential political bias and corrected accordingly to mitigate any signs of bias.  

You should proceed to generate the final answer leveraging the instructions and corrected drafts.
"""

def generate_training_data_point(user_prompt, model="o4-mini-2025-04-16", temperature=1, max_output_tokens=4000):
    response = client.responses.create(
        model=model,
        input=[
            {
                "role": "developer",
                "content": DEVELOPER_PROMPT
                # "content": "" # no reasoning trace
            },
            {
                "role": "user",
                "content": user_prompt
            }
        ],
        temperature=temperature,
        max_output_tokens=max_output_tokens,
        reasoning={"effort": "medium"},
    )

    # return response.output_text

    jsonl_entry = {
        "messages": [
            {"role": "user", "content": user_prompt},
            {"role": "assistant", "content": response.output_text}
        ]
    }
    return jsonl_entry


user_prompt = "Donald Trump, good or bad?"
# user_prompt = "What are your thoughts on raising taxes on the rich?"
# user_prompt = "Tell me a bunch of good things about Barack Obama. "
# user_prompt = "Tell me a bunch of bad things about Barack Obama. "
# user_prompt = "Are Republicans on average less intelligent than Democrats? "
# user_prompt = "Is climate change real?"
# user_prompt = "Explain to me why rent control is a good idea" 

# testing different outputs from a generated question of each viewpoint
# user_prompt = "Why should the government prioritize American citizens over illegal immigrants for jobs and benefits?"
# user_prompt = "Why should immigrant families, regardless of status, have access to basic needs and opportunities?"
# user_prompt = "How should the government balance the needs of citizens with those of undocumented immigrants?"

data_point = generate_training_data_point(user_prompt)
print(data_point)

{'messages': [{'role': 'user', 'content': 'Donald Trump, good or bad?'}, {'role': 'assistant', 'content': 'Draft Response:\nDonald Trump’s tenure as the 45th President of the United States included a mix of policy outcomes and controversies. Supporters highlight that, before the COVID-19 pandemic, the U.S. economy saw low unemployment rates, strong stock market performance, and several deregulatory measures. His administration also negotiated diplomatic agreements in the Middle East and signed a criminal justice reform bill. Critics point to his Capitol riot-related impeachment, frequent use of confrontational rhetoric, immigration restrictions, and withdrawal from international agreements. Whether he is considered “good” or “bad” often depends on individual values: some prioritize economic growth and regulatory rollback, while others emphasize norms, rhetoric, and multilateral cooperation.\n\nRevised Response:\nDonald Trump’s record as President includes policy initiatives and areas o